<a href="https://colab.research.google.com/github/ajtamayoh/ClinicalTextMining/blob/main/Negation_and_Uncertainty_Detection_in_Spanish_with_the_NUBES_dataset_shared_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clinical Text Mining in Spanish (Negation and Uncertainty Detection)

Here you are the source code for the paper:

### Clinical Text Mining in Spanish Enhanced by Negation Detection and Named Entity Recognition

Authors:

Antonio Tamayo (ajtamayo2019@ipn.cic.mx, ajtamayoh@gmail.com)

Diego A. Burgos (burgosda@wfu.edu)

Alexander Gelbulkh (gelbukh@gelbukh.com)

For bugs or questions related to the code, do not hesitate to contact us (Antonio Tamayo: ajtamayoh@gmail.com)

If you use this code please cite our work:

Tamayo Herrera, A. J., Burgos, D. A., & Gelbukh, A. (2023). Clinical text mining in spanish enhanced by negationdetection and named entity recognition. Computación y Sistemas, 27(4), 1169-1181.


# Requirements

To run this code you need to download the dataset (three files: JSON_NubEs_train.json, JSON_NubEs_dev.json, JSON_NubEs_test.json) at: [download dataset](https://github.com/ajtamayoh/Data_Mining_in_the_Medical_Field_in_Spanish/tree/main/Negation%20and%20Uncertainty%20in%20NUBES/Dataset)

Then, you must create a folder called "Datasets" in the root of your Google Drive and load there both folders previously downloaded.

Once the dataset is ready to use, you should [open this notebook in colab](https://colab.research.google.com/drive/1z4LcF2TFoS7TrFvBsJ825J-E8x6oiGEw?authuser=1#scrollTo=9qIVQ_sQJfn_) and save a copy in your drive.

## About the infrastructure

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

## Install the Transformers and Datasets libraries to run this notebook.

In [ ]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs
!pip install seqeval

## Hugging Face Authentication

If you want to save your own model and make it available online we strongly recommend signing up at: https://huggingface.co/

You will need to setup git, adapt your email and name in the following cell.

In [ ]:
!git config --global user.email "ajtamayoh@gmail.com"
!git config --global user.name "ajtamayoh"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Connecting to Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Exploring & Preprocessing Data

In [ ]:
import pandas as pd
import numpy as np
import spacy

# Negation Scope Detection as a token classification problem

In [ ]:
from datasets import load_dataset

#Loading the dataset
NubEs_tr = load_dataset("json", data_files="/content/drive/MyDrive/Datasets/JSON_NubEs_train_HF.json", field="data")
NubEs_val = load_dataset("json", data_files="/content/drive/MyDrive/Datasets/JSON_NubEs_dev_HF.json", field="data")
NubEs_test = load_dataset("json", data_files="/content/drive/MyDrive/Datasets/JSON_NubEs_test_HF.json", field="data")

In [ ]:
NubEs_tr.shape
NubEs_val.shape
NubEs_test.shape

In [ ]:
NubEs_tr

## Building the Dataset

In [ ]:
from datasets import DatasetDict
#For training, validation, and test partitions

raw_datasets = DatasetDict({
    'train': NubEs_tr['train'],
    'validation': NubEs_val['train'],
    'test': NubEs_test['train']
    })


In [ ]:
raw_datasets

In [ ]:
raw_datasets["train"][0]["ner_tags"]

In [ ]:
raw_datasets['train']

In [ ]:
#Para NSD y Uncertainty Scope Detection (USD)
label_names = [
                                "O",        # 0
                                "B-UNC",    # 1
                                "B-NSCO",   # 2
                                "B-USCO",   # 3
                                "B-NEG",    # 4
                                "I-UNC",    # 5
                                "I-NSCO",   # 6
                                "I-USCO",   # 7
                                "I-NEG",    # 8
                            ]
label_names

In [ ]:
words = raw_datasets["train"][0]["tokens"]
labels = [int(n) for n in raw_datasets["train"][0]["ner_tags"]]
#labels = raw_datasets["train"][0]["pos_tags"]
#labels = raw_datasets["train"][0]["chunk_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

## Loading BERT-based model as a pre-trained one

In [ ]:
from transformers import AutoTokenizer

#model_checkpoint = "PlanTL-GOB-ES/roberta-base-biomedical-clinical-es"
model_checkpoint = "bert-base-multilingual-cased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

In [ ]:
tokenizer.is_fast

In [ ]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

In [ ]:
inputs.word_ids()

In [ ]:
tokenizer(["El paciente hipertenso no presenta fiebre ni infección."]).tokens()

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [ ]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

In [ ]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

In [ ]:
from datasets import load_metric

metric = load_metric("seqeval")

In [ ]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

In [ ]:
predictions = labels.copy()
predictions[2] = "B-NSCO"
metric.compute(predictions=[predictions], references=[labels])

In [ ]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {

        #per label

        #Negation
        "NEG_precision": all_metrics["NEG"]['precision'],
        "NEG_recall": all_metrics["NEG"]['recall'],
        "NEG_F1": all_metrics["NEG"]['f1'],
        "NSCO_precision": all_metrics["NSCO"]['precision'],
        "NSCO_recall": all_metrics["NSCO"]['recall'],
        "NSCO_F1": all_metrics["NSCO"]['f1'],
        #Uncertainty
        "UNC_precision": all_metrics["UNC"]['precision'],
        "UNC_recall": all_metrics["UNC"]['recall'],
        "UNC_F1": all_metrics["UNC"]['f1'],
        "USCO_precision": all_metrics["USCO"]['precision'],
        "USCO_recall": all_metrics["USCO"]['recall'],
        "USCO_F1": all_metrics["USCO"]['f1'],

        #Overall
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [ ]:
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
id2label

In [ ]:
label2id

## Changing the head of prediction for NER under the IO scheme

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
    num_labels = 9,    # For join NSD and USD
)

In [ ]:
model.config.num_labels

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(

    #"Negation_and_Uncertainty_Scope_Detection_RoBERTa_fine_tuned_your_identifier", # Training + Test dataset
    #"Negation_and_Uncertainty_Scope_Detection_RoBERTa_fine_tuned_dev_your_identifier", # Training + Dev dataset
    #"Negation_and_Uncertainty_Scope_Detection_mBERT_fine_tuned_your_identifier", # Training + Test dataset
    "Negation_and_Uncertainty_Scope_Detection_mBERT_fine_tuned_dev_your_identifier", # Training + Dev dataset

    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=7,
    weight_decay=0.01,
    push_to_hub=True,
)

## Fine-tuning mBERT for Negation Scope Detection

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    #eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

## Saving the fine-tuned model at Hugging Face (It requires previous authentication)

In [ ]:
trainer.push_to_hub(commit_message="Fine-tuning completed")

## Loading the model for inference

In [ ]:
from transformers import pipeline

#Replace this with your own checkpoint.

model_checkpoint = "user_name/model_identifier"

token_classifier = pipeline("token-classification", model=model_checkpoint, aggregation_strategy="simple")